In [10]:
from pathlib import Path

import cv2
import numpy as np
import rasterio
from tqdm import tqdm
from skimage import morphology

In [11]:
def run(input_raster, output_raster):
    src = rasterio.open(input_raster)
    src_img = src.read(1)
    profile = src.meta.copy()
    profile.update({'compress': 'deflate'})
    src.close()
    # 连通组件分析

    kernel = morphology.square(3)  # 3*3的正方形腐蚀核
    img_dilated = morphology.dilation(src_img, kernel)  # 膨胀
    edge_dilated = img_dilated - src_img  # 膨胀后的图像减去腐蚀后的图像，得到边缘
    edge_dilated = morphology.skeletonize(edge_dilated)
    xs_, ys_ = np.where(edge_dilated == 1)

    # 腐蚀后计算连通域
    _, labels_eroded, _, _ = cv2.connectedComponentsWithStats(src_img, connectivity=8)

    with tqdm(range(len(xs_)), desc='Progress') as tbar:
        # 对于每一个可能的关键点
        for x, y in zip(xs_, ys_):
            # 获取该点在腐蚀图像中的邻域
            neighborhood = set(labels_eroded[x - 1:x + 2, y - 2:y + 3].reshape(-1))
            neighborhood.update(labels_eroded[x - 2, y - 1:y + 2].reshape(-1))
            neighborhood.update(labels_eroded[x + 2, y - 1:y + 2].reshape(-1))

            # 如果邻域包含不同的连通域
            if len(neighborhood) > 2:  # 包括背景0
                # 那么这个点就可能是关键点
                img_dilated[x, y] = 0

            tbar.update()

    with rasterio.open(output_raster, 'w', **profile) as dst:
        dst.write(img_dilated, 1)


In [12]:
# 设置目录路径和文件后缀
dir_path = Path(r'D:\UAV_DATA_NEW\train_sample')
output_path = Path(r'D:\UAV_DATA_NEW\train_sample_dia')

# 循环处理每个文件
for input_raster in dir_path.glob(f'*.tif'):
    file_name = input_raster.stem[:6]
    output_raster = output_path / f'{file_name}_dilated.tif'
    print(output_raster.name)
    run(input_raster, output_raster)


061301_dilated.tif


Progress: 100%|██████████| 144262/144262 [00:00<00:00, 274698.01it/s]


061302_dilated.tif


Progress: 100%|██████████| 252202/252202 [00:00<00:00, 289575.21it/s]


061303_dilated.tif


Progress: 100%|██████████| 259054/259054 [00:00<00:00, 272059.29it/s]
